In [3]:
# ────────────────────────────── installs ──────────────────────────────
# !pip install -qU pandas scikit-learn torch transformers datasets

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    TrainingArguments, Trainer, pipeline
)

# ═══════════════════ 1. Load & filter dataset ═════════════════════════
df = pd.read_csv("cyberbullying_tweets_cleaned.csv")

wanted = ["age", "religion"]
df = df[df["cyberbullying_type"].isin(wanted)].copy()

label_map  = {"age": 0, "religion": 1}
df["label"] = df["cyberbullying_type"].map(label_map)

print(df["label"].value_counts())   # check class balance

# ═══════════════════ 2. Train / val split ════════════════════════════
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["cleaned_text"].astype(str).tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

# ═══════════════════ 3. Tokenise ═════════════════════════════════════
tok = BertTokenizer.from_pretrained("bert-base-uncased")

train_enc = tok(train_texts, truncation=True, padding=True, max_length=128)
val_enc   = tok(val_texts,   truncation=True, padding=True, max_length=128)

class CyberDS(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.labels[idx])
        }


train_ds = CyberDS(train_enc, train_labels)
val_ds   = CyberDS(val_enc,   val_labels)

# ═══════════════════ 4. Model with correct label names ═══════════════
id2label = {0: "age", 1: "religion"}
label2id = {v: k for k, v in id2label.items()}

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    problem_type="single_label_classification"
)

# ═══════════════════ 5. Training setup ═══════════════════════════════
args = TrainingArguments(
    output_dir="./cyberbully_results",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tok,
)

trainer.train()
metrics = trainer.evaluate()
print("Validation metrics:", metrics)

# After trainer.train()
torch.save(model.state_dict(), "bert_cyberbullying.pth")


# ═══════════════════ 6. Save fine-tuned model ════════════════════════
SAVE_DIR = "bert-cyberbullying"
model.save_pretrained(SAVE_DIR)
tok.save_pretrained(SAVE_DIR)

# ═══════════════════ 7. Quick inference demo ════════════════════════
clf = pipeline(
    "text-classification",
    model=SAVE_DIR,
    tokenizer=SAVE_DIR,
    return_all_scores=True
)

sample = "Go back to the nursery, kid!"
print("\nSample :", sample)
print("Scores :", clf(sample))
#Replace this code from your current model code

label
1    7998
0    7992
Name: count, dtype: int64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-eef013bfc1c7>:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jkos0012 (jkos0012-monash-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.076600,0.008448
2,0.001000,0.015388
3,0.000000,0.009727
4,0.000000,0.012758


Validation metrics: {'eval_loss': 0.008448093198239803, 'eval_runtime': 14.1312, 'eval_samples_per_second': 226.307, 'eval_steps_per_second': 7.077, 'epoch': 4.0}


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(



Sample : Go back to the nursery, kid!
Scores : [[{'label': 'age', 'score': 0.9958167672157288}, {'label': 'religion', 'score': 0.0041832467541098595}]]
